In [40]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent
sys.path.append(str(Path(os.getcwd()).parent))

In [41]:
from api import GeoCoader
import pandas as pd
import re
from datetime import datetime

In [82]:
# 파일 임의로 수정함
path = os.path.join(root, 'data', 'modified_route3.csv')
route = pd.read_csv(path, encoding='utf-8-sig')
route

,date,patNo,reason,route,type,latitude,longitude
0,2020-04-17,10779,해외 접촉,미국 LA 출발 아시아나항공(OZ201) 이용해 인천공항에 입국,airport,33.941589,-118.408530
1,2020-05-01,10789,해외 접촉,"인천국제공항(대한항공 KE8956, 좌석 55C)",airport,37.460191,126.440696
2,2020-05-01,10789,해외 접촉,공항,airport,37.442055,127.108156
3,2020-04-28,10790,해외 접촉,인천공항 입국(미국),airport,37.460191,126.440696
4,2020-05-02,10839,이태원 클럽 관련,"킹클럽(우사단로, 클럽), 퀸(우사단로, 주점)",bar,37.533634,126.995761
...,...,...,...,...,...,...,...
3235,2020-08-08,15529,성북구 사랑제일교회 관련,"마트(강북구 오현로), 마스크 착용, 접촉자 없음, 소독완료",store,37.619431,127.034972
3236,2020-08-13,15565,성북구 사랑제일교회 관련,"병원(강북구 도봉로), 마스크 착용, 접촉자 없음, 소독완료",hospital,37.670341,127.040780
3237,2020-08-13,15565,성북구 사랑제일교회 관련,"유통업(강북구 도봉로), 마스크 착용, 접촉자 없음, 소독완료",etc,37.631575,127.027822
3238,2020-08-12,15564,성북구 사랑제일교회 관련,"외식업(강북구 한천로), 마스크 착용, 접촉자 없음, 소독완료",point_of_interest,37.633069,127.036275


In [83]:
place_types = route['type']
place_types = place_types.unique()
place_types

array(['airport', 'bar', 'pc_café', 'cafe', 'food', 'convenience_store',
       'point_of_interest', 'gym', 'restaurant', 'subway', 'karaoke',
       'bus_station', 'doctor', 'pharmacy', 'administrative_area_level_1',
       'night_club', 'store', 'atm', 'hospital', 'study', 'laundry',
       'department_store', 'park', 'dorm', 'school', 'church', 'bank',
       'car_repair', 'academy', 'work', 'gas_station', 'sauna',
       'welfare_center', 'studio', 'temple', 'bakery', 'door_sales',
       'beauty_salon', 'etc', 'real_estate', 'lodging', 'post_office'],
      dtype=object)

In [81]:
i = 5
bar = route[route['type'] == place_types[i]]
bar

,date,patNo,reason,route,type,latitude,longitude
45,2020-05-04,10840,이태원 클럽 관련,별밤포차(한강대로),point_of_interest,37.544065,126.971637
58,2020-05-02,10827,이태원 클럽 관련,지하철 이용 낙성대역 하차,point_of_interest,37.477117,126.963418
59,2020-05-03,10827,이태원 클럽 관련,지하철 이용 낙성대역,point_of_interest,37.477117,126.963418
61,2020-05-03,10827,이태원 클럽 관련,지하철 2호선 낙성대역 도착 신림역으로 이동,point_of_interest,37.484269,126.929676
62,2020-05-04,10827,이태원 클럽 관련,솟을대문 신림역점,point_of_interest,37.485078,126.928195
...,...,...,...,...,...,...,...
3268,2020-08-11,14842,성북구 사랑제일교회 관련,"약국(강북구 한천로), 마스크 착용, 접촉자 없음, 소독완료",point_of_interest,37.633069,127.036275
3269,2020-08-12,14842,성북구 사랑제일교회 관련,"관공서(강북구 한천로), 마스크 착용, 접촉자 없음, 소독완료",point_of_interest,37.633069,127.036275
3280,2020-08-13,15110,성북구 사랑제일교회 관련,"약국(강북구 한천로), 마스크 착용, 접촉자 없음, 소독완료",point_of_interest,37.633069,127.036275
3301,2020-08-12,15564,성북구 사랑제일교회 관련,"외식업(강북구 한천로), 마스크 착용, 접촉자 없음, 소독완료",point_of_interest,37.633069,127.036275


In [7]:
wrong_tags = ['administrative_area_level_1', 'point_of_interest', 'sublocality', 'establishment', 'political', 'street_address', 'accounting', 'subpremise', 'locality']

In [38]:
geo_api = GeoCoader()

indices_to_del = []
count = 0
for tag in wrong_tags:
    print('tag: %s' % tag)
    places = route[route['type'] == tag]
    for index, row in places.iterrows():
        if count  % 100 == 0: print(count)
        count += 1
        print(row['route'])
        information = geo_api.get_information(row['route'])
        if information.empty:
            indices_to_del.append(index)
            continue
        route.at[index, 'type'] = information['type'].values[0]
        
    print(len(indices_to_del))

CU서울대서연점(관악로 132) 


IndexError: list index out of range

path = os.path.join(root, 'data', 'modified_route3.csv')
route.to_csv(path, encoding='utf-8-sig', index=False)